Trying to predict if a pair of compounds is a MAIN pair

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/pairs_final_RPAIRS_smiles.csv', index_col=0)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,RPAIR_main
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# keep rows with 'RPAIR_main' !=2
df_train = df[df['RPAIR_main'] != 2]
df_test = df[df['RPAIR_main'] == 2]

print(df_train.shape, df_test.shape)

(21071, 2049) (34909, 2049)


In [5]:
# split df_train into train and validation
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)